<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/auduvignac/challengedata_ens_AssurPrime/blob/main/notebooks/exploration/exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

##Installation de category-encoder

In [1]:
!pip install category-encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.6 MB/s eta 0:00:00


##Importation des bibliothèques

In [2]:
# Importer les bibliothèques nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
pd.set_option('display.max_columns', 375)
import numpy as np
from category_encoders import CountEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

##Chargement des données

In [3]:
# Charger les données
try:
  print("Chargement des données...")
  X_train_url = "https://media.githubusercontent.com/media/auduvignac/challengedata_ens_AssurPrime/refs/heads/main/data/raw/x_train.csv"
  X_train = pd.read_csv(X_train_url)
  y_train_url = "https://media.githubusercontent.com/media/auduvignac/challengedata_ens_AssurPrime/refs/heads/main/data/raw/y_train.csv"
  y_train = pd.read_csv(y_train_url)
  X_test_url = "https://media.githubusercontent.com/media/auduvignac/challengedata_ens_AssurPrime/refs/heads/main/data/raw/x_test.csv"
  X_test = pd.read_csv(X_test_url)
  print("Données chargées avec succès.")
except:
  print("Erreur lors du chargement des données. Vérifiez que les URLs sont correctes.")

Chargement des données...


<ipython-input-3-c5aaf42838da>:5: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv(X_train_url)
<ipython-input-3-c5aaf42838da>:9: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv(X_test_url)


Données chargées avec succès.


In [4]:
# Traitement des valeurs manquantes dans les colonnes numériques
print("Traitement des valeurs manquantes dans les colonnes numériques...")
numeric_columns = X_train.select_dtypes(include=['number']).columns

# Remplir les NaN avec 0 pour les colonnes numériques
X_train[numeric_columns] = X_train[numeric_columns].fillna(0)

# Identifier les colonnes non numériques
fill_cols = [item for item in X_train.columns if item not in numeric_columns]

# Remplir les NaN des colonnes non numériques avec une valeur par défaut (-999)
X_train[fill_cols] = X_train[fill_cols].fillna(-999)

print("Traitement des valeurs manquantes terminé.")

Traitement des valeurs manquantes dans les colonnes numériques...
Traitement des valeurs manquantes terminé.


In [5]:
# Préparation des données pour l'entraînement
print("Préparation des données pour l'entraînement...")

# Suppression des colonnes inutiles
X_train = X_train.drop(['ID', 'ANNEE_ASSURANCE'], axis=1)

# Encodage des variables catégoriques avec CountEncoder
encoder = CountEncoder(cols=fill_cols)
encoder.fit(X_train)
X_train_enc = encoder.transform(X_train)

print("Préparation terminée.")

Préparation des données pour l'entraînement...
Préparation terminée.


###Insertion de code


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, fbeta_score, mean_squared_error
from xgboost import XGBClassifier, XGBRegressor
import pandas as pd
import numpy as np

In [7]:
def train_classifier(X, y_binary, scale_pos_weight=99):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    clf = XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        n_jobs=-1,
        verbosity=0,
        scale_pos_weight=scale_pos_weight
    )
    clf.fit(X_scaled, y_binary)
    return clf, scaler

In [8]:
def find_best_threshold(y_true, y_proba, beta=2, thresholds=np.linspace(0.01, 0.99, 100)):
    best_thresh, best_score = 0, 0
    for t in thresholds:
        y_pred = (y_proba >= t).astype(int)
        score = fbeta_score(y_true, y_pred, beta=beta)
        if score > best_score:
            best_thresh, best_score = t, score
    return best_thresh, best_score


In [9]:
def train_regressors(X, y_freq, y_cm, mask):
    X_sub = X[mask]
    y_freq_sub = y_freq[mask]
    y_cm_sub = y_cm[mask]

    reg_freq = XGBRegressor(n_jobs=-1, verbosity=0)
    reg_cm = XGBRegressor(n_jobs=-1, verbosity=0)

    reg_freq.fit(X_sub, y_freq_sub)
    reg_cm.fit(X_sub, y_cm_sub)

    return reg_freq, reg_cm


In [10]:
def run_pipeline(X, y, beta=2):
    # Étape 1 : classification
    y_binary = (y['FREQ'] > 0).astype(int)
    clf, scaler = train_classifier(X, y_binary)

    X_scaled = scaler.transform(X)
    y_proba = clf.predict_proba(X_scaled)[:, 1]

    threshold, fbeta = find_best_threshold(y_binary, y_proba, beta=beta)
    print(f"Meilleur seuil (F{beta}): {threshold:.2f} - Score: {fbeta:.3f}")

    y_pred_class = (y_proba >= threshold).astype(int)
    print(classification_report(y_binary, y_pred_class, digits=3))

    # Étape 2 : entraînement des régressions sur les cas classés 1
    mask_classified_pos = y_pred_class == 1
    reg_freq, reg_cm = train_regressors(X, y['FREQ'], y['CM'], mask_classified_pos)

    return clf, scaler, threshold, reg_freq, reg_cm


In [11]:
clf, scaler, threshold, reg_freq, reg_cm = run_pipeline(X_train_enc, y_train)

Meilleur seuil (F2): 0.73 - Score: 0.450
              precision    recall  f1-score   support

           0      0.997     0.987     0.992    380716
           1      0.249     0.565     0.345      2894

    accuracy                          0.984    383610
   macro avg      0.623     0.776     0.669    383610
weighted avg      0.991     0.984     0.987    383610



In [13]:
def predict_pipeline(X, clf, scaler, threshold, reg_freq, reg_cm):
    X_scaled = scaler.transform(X)
    y_proba = clf.predict_proba(X_scaled)[:, 1]
    mask = y_proba >= threshold

    freq_pred = np.zeros(X.shape[0])
    cm_pred = np.zeros(X.shape[0])

    if mask.sum() > 0:
        freq_pred[mask] = reg_freq.predict(X[mask])
        cm_pred[mask] = reg_cm.predict(X[mask])

    return freq_pred, cm_pred, y_proba


In [15]:
freq_pred, cm_pred, infl_proba = predict_pipeline(X_train_enc, clf, scaler, threshold, reg_freq, reg_cm)


###Evaluation possible

In [19]:
from sklearn.metrics import mean_squared_error
import numpy as np

mask_true = (y_train['FREQ'] > 0)

# Calcul du MSE puis de la racine
mse_freq = mean_squared_error(y_train['FREQ'][mask_true], freq_pred[mask_true])
rmse_freq = np.sqrt(mse_freq)

mse_cm = mean_squared_error(y_train['CM'][mask_true], cm_pred[mask_true])
rmse_cm = np.sqrt(mse_cm)


In [24]:
from sklearn.metrics import mean_squared_error

rmse_freq = mean_squared_error(y_train['FREQ'][mask_true], freq_pred[mask_true])
rmse_cm   = mean_squared_error(y_train['CM'][mask_true], cm_pred[mask_true])

print(f"RMSE FREQ (cond. FREQ > 0) : {rmse_freq:.3f}")
print(f"RMSE CM   (cond. FREQ > 0) : {rmse_cm:.3f}")


RMSE FREQ (cond. FREQ > 0) : 2.252
RMSE CM   (cond. FREQ > 0) : 2185840122.225


###Fin d'insertion de code

###Calcul de la prédiction combinée

In [25]:
# Calculer la prédiction combinée pour 'CHARGE'
y_train_pred = freq_pred * cm_pred * y_train['ANNEE_ASSURANCE']

In [26]:
# Calculer le RMSE sur l'ensemble d'entraînement
rmse = np.sqrt(mean_squared_error(y_train['CHARGE'], y_train_pred))
print(f"RMSE sur l'ensemble d'entraînement : {rmse:.2f}")

RMSE sur l'ensemble d'entraînement : 4642.90
